In [247]:
def infoGain(P,N):
    import math
    return -P/(P+N)*math.log2(P/(P+N))-N/(P+N)*math.log2(N/(P+N))

In [248]:
def countConcept(data,attr,val):
    from collections import Counter
    cnt=Counter(x for x in data[attr])
    return cnt[val]

In [249]:
def countAttr(data,val,attr,concept,cVal):
    count={}
    C = data[concept]
    A = data[val]
    for a in data.index:
        for v in attr:
            if v not in count:
                count[v] = 0
            if(A[a]==v and C[a]==cVal):
                if v not in count:
                    count[v]=1
                else:
                    count[v]=count[v]+1
    return count

In [250]:
def insertNode(tree,d,Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertNode(v, d, Node)
    if d in tree:
        if isinstance(tree[d], dict):
            tree[d][Node] = 'None' #Problem Here
        else:
            tree[d] = {Node:'None'} 
    return tree

In [251]:
def insertConcept(tree,d,Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertConcept(v, d, Node)
    if d in tree:
        tree[d] = Node
    return tree

In [252]:
def getNextData(data,Node,d):
    iData = (data[Node] == d)
    nD = data[iData]
    nD = nD.drop(Node,axis=1)
    return nD

In [256]:
def getNextNode(data,AttributeList,concept,conceptVals,tree,d):
    Total=data.shape[0]
    if Total == 0: #If no Attibute then return tree
        return tree
    
    countC={}
    for cVal in conceptVals: #If Example is positive then return positive and If negative then return negative
        countC[cVal] = countConcept(data,concept,cVal)
        if countC[cVal] == 0 and cVal == 'No':
            tree = insertConcept(tree,d,'Yes')
            return tree
        if countC[cVal] == 0 and cVal == 'Yes':
            tree = insertConcept(tree,d,'No')
            return tree
    
    ClassEntropy = infoGain(countC['Yes'],countC['No']) #Class Entropy for d
    
    Attr = {}#Attribute dict holding list of possible values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))

    AttrCount = {} #Get the attribute values being positive and negative
    for val in Attr:
        AttrCount[val] = {}
        for cVal in conceptVals:
            AttrCount[val][cVal] = countAttr(data,val,Attr[val],concept,cVal)

    EntropyAttr = {} #Calculate Attribute entropy
    AttPosNeg = {}
    for att in AttrCount:
        InfoGain = {}
        ap = {}
        an = {}
        for c in conceptVals:
            for val in AttrCount[att][c]:
                if c == 'No':
                    an[val] = AttrCount[att][c][val]
                if c == 'Yes':
                    ap[val] = AttrCount[att][c][val]
        AttPosNeg[att]={}
        TotalInfo = {}
        AttPosNeg[att] = {'Yes':ap,'No':an}
        for val in AttrCount[att][c]:
            TotalInfo[val] = ap[val]+an[val]
            if ap[val]==0 or an[val]==0:
                InfoGain[val]=0
            else:
                InfoGain[val] = infoGain(ap[val],an[val])
#             print("Information Gain : ",InfoGain)
        for val in InfoGain:
            if att not in EntropyAttr:
                EntropyAttr[att] = ( TotalInfo[val] / Total ) * InfoGain[val]
            else:
                EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo[val] / Total ) * InfoGain[val]
#         print("Attribute Entropy : ",EntropyAttr)
    Gain = {}
    for g in EntropyAttr:
        if g not in Gain:
            Gain[g] = ClassEntropy - EntropyAttr[g]
#         print("Gain : ",Gain)
    Node = max(Gain,key=Gain.get)
#     print("Next root : ",Node)
    tree = insertNode(tree,d,Node)
    for nD in Attr[Node]:
        tree = insertNode(tree,Node,nD)
        newData = getNextData(data,Node,nD)
#         print(nD,newData)
        AttributeList=list(newData)[:-1]
        tree = getNextNode(newData,AttributeList,concept,conceptVals,tree,nD)
#         print(tree)  
    return tree

In [259]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('play.csv')
    AttributeList=list(data)[:-1]
    concept=str(list(data)[-1])
    conceptVals = list(set(data[concept]))
    tree = getNextNode(data,AttributeList,concept,conceptVals,{'root':'None'},'root')
    print(tree)

In [260]:
main()

{'root': {'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}}}}
